In [ ]:
case_ID = 0

from pathlib import Path
import gmsh
from mpi4py import MPI
from dolfinx.io import gmshio
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.tri import Triangulation

gmsh.initialize()
gmsh.model.add("geometry")

L1, L2, L3, L4 = 4., 4., 4., 4.
h, w = 4., 4.

meshsize = 0.25

p1 = gmsh.model.geo.addPoint(-L1/2, -L4/2, 0, meshSize=meshsize)
p2 = gmsh.model.geo.addPoint(0, -h/2, 0, meshSize=meshsize)
p3 = gmsh.model.geo.addPoint(L1/2, -L2/2, 0, meshSize=meshsize)
p4 = gmsh.model.geo.addPoint(w/2, 0, 0, meshSize=meshsize)
p5 = gmsh.model.geo.addPoint(L3/2, L2/2, 0, meshSize=meshsize)
p6 = gmsh.model.geo.addPoint(0, h/2, 0, meshSize=meshsize)
p7 = gmsh.model.geo.addPoint(-L3/2, L4/2, 0, meshSize=meshsize)
p8 = gmsh.model.geo.addPoint(-w/2, 0, 0, meshSize=meshsize)

l1 = gmsh.model.geo.add_bspline([p1, p2, p3])
l2 = gmsh.model.geo.add_bspline([p3, p4, p5])
l3 = gmsh.model.geo.add_bspline([p5, p6, p7])
l4 = gmsh.model.geo.add_bspline([p7, p8, p1])

curve_loop = gmsh.model.geo.addCurveLoop([l1, l2, l3, l4])
surface = gmsh.model.geo.addPlaneSurface([curve_loop])

gmsh.model.geo.mesh.setTransfiniteCurve(l1, 10)
gmsh.model.geo.mesh.setTransfiniteCurve(l2, 10)
gmsh.model.geo.mesh.setTransfiniteCurve(l3, 10)
gmsh.model.geo.mesh.setTransfiniteCurve(l4, 10)
gmsh.model.geo.mesh.setTransfiniteSurface(surface)

gmsh.model.geo.synchronize()

bottom_tag_num, left_tag_num, top_tag_num, right_tag_num, interior_tag_num = 1, 2, 3, 4, 5

gmsh.model.addPhysicalGroup(1, [l1], bottom_tag_num, name="bottom")
gmsh.model.addPhysicalGroup(1, [l2], left_tag_num, name="left")
gmsh.model.addPhysicalGroup(1, [l3], top_tag_num, name="top")
gmsh.model.addPhysicalGroup(1, [l4], right_tag_num, name="right")
gmsh.model.addPhysicalGroup(2, [surface], interior_tag_num, name="interior")

gmsh.model.mesh.generate(2)

mesh_folder = Path("./paraview") / f"case_ID_{case_ID}"
mesh_folder.mkdir(exist_ok=True, parents=True)
gmsh.write(str(mesh_folder) + "/mesh.vtk")

domain, cell_tags, facet_tags = gmshio.model_to_mesh(gmsh.model, MPI.COMM_WORLD, 0, gdim=2)

gmsh.finalize()

points = domain.geometry.x
cells = domain.topology.connectivity(2, 0).array.reshape((-1, 3))

# Plot the mesh using Matplotlib
plt.figure(figsize=(8, 6))
triang = Triangulation(points[:, 0], points[:, 1], cells)
plt.triplot(triang, color="black")
plt.gca().set_aspect('equal')
plt.title("2D Geometry Mesh")
plt.xlabel("$x_1$")
plt.ylabel("$x_2$")
plt.show()

In [ ]:
class exact_solution():
    def __init__(self, alpha):
        self.alpha = alpha

    def __call__(self, x):
        return 1 + x[0]**2 * x[1] + self.alpha * x[0] * x[1]**2
    
u_exact = exact_solution(alpha=2)

In [ ]:
from dolfinx import fem, default_scalar_type
import ufl

V2 = fem.functionspace(domain, ("Lagrange", 2))

u_ex2 = fem.Function(V2)
u_ex2.interpolate(u_exact)

tdim = domain.topology.dim
fdim = tdim - 1
domain.topology.create_connectivity(fdim, tdim)

dofs_bottom_b = fem.locate_dofs_topological(V2, fdim, facet_tags.find(bottom_tag_num))
dofs_left_b = fem.locate_dofs_topological(V2, fdim, facet_tags.find(left_tag_num))
dofs_top_b = fem.locate_dofs_topological(V2, fdim, facet_tags.find(top_tag_num))
dofs_right_b = fem.locate_dofs_topological(V2, fdim, facet_tags.find(right_tag_num))

dofs_drchlt_BC = np.concatenate([dofs_left_b, dofs_right_b])
dofs_neumnn_BC = np.concatenate([dofs_bottom_b, dofs_top_b])

f_left = fem.Function(V2)
# f_left.interpolate(lambda x: np.full(x.shape[1], 0))
f_left.interpolate(u_exact)

f_right = fem.Function(V2)
# f_right.interpolate(lambda x: np.full(x.shape[1], 0))
f_right.interpolate(u_exact)

drchlt_bc_left = fem.dirichletbc(f_left, dofs_left_b)
drchlt_bc_right = fem.dirichletbc(f_right, dofs_right_b)

In [ ]:
x = ufl.SpatialCoordinate(domain)
src_term = - (2 * u_exact.alpha * x[0] + 2 * x[1])

g_bottom = (x[0]**2 + 2 * u_exact.alpha * x[0] * x[1])
g_top = -(x[0]**2 + 2 * u_exact.alpha * x[0] * x[1])

ds = ufl.Measure("ds", domain=domain, subdomain_data=facet_tags)

In [ ]:
u, v = ufl.TrialFunction(V2), ufl.TestFunction(V2)
a = ufl.dot(ufl.grad(u), ufl.grad(v)) * ufl.dx
L = src_term * v * ufl.dx - g_bottom * v * ds(bottom_tag_num) - g_top * v * ds(top_tag_num)

bilinear_form = fem.form(a)
linear_form = fem.form(L)

In [ ]:
from dolfinx.fem.petsc import assemble_vector, assemble_matrix, \
    create_vector, apply_lifting, set_bc
from petsc4py import PETSc

A = assemble_matrix(bilinear_form, bcs=[drchlt_bc_left, drchlt_bc_right])
A.assemble()
b = create_vector(linear_form)

with b.localForm() as loc_b:
    loc_b.set(0)
assemble_vector(b, linear_form)
apply_lifting(b, [bilinear_form], [[drchlt_bc_left, drchlt_bc_right]])
b.ghostUpdate(addv=PETSc.InsertMode.ADD_VALUES, mode=PETSc.ScatterMode.REVERSE)
set_bc(b, [drchlt_bc_left, drchlt_bc_right])

uh2 = fem.Function(V2)

solver = PETSc.KSP().create(domain.comm)
solver.setOperators(A)
solver.setType(PETSc.KSP.Type.GMRES)
solver.getPC().setType(PETSc.PC.Type.ILU) 
solver.setTolerances(rtol=1e-12)

def monitor(ksp, its, rnorm):
    print(f"Iteration {its}, Residual norm {rnorm}")

solver.setMonitor(monitor)
solver.solve(b, uh2.x.petsc_vec)

In [ ]:
V1 = fem.functionspace(domain, ("Lagrange", 1))
u_ex1 = fem.Function(V1)
u_ex1.interpolate(u_exact)

uh1 = fem.Function(V1)
uh1.interpolate(uh2)

L2_error = fem.form(ufl.inner(uh1 - u_ex1, uh1 - u_ex1) * ufl.dx)
error_local = fem.assemble_scalar(L2_error)
error_L2 = np.sqrt(domain.comm.allreduce(error_local, op=MPI.SUM))

error_max = np.max(np.abs(u_ex1.x.array-uh1.x.array))
if domain.comm.rank == 0:
    print(f"Error_L2 : {error_L2:.2e}, Error_max : {error_max:.2e}")

In [ ]:
error_L2 = np.sqrt(domain.comm.allreduce(fem.assemble_scalar(fem.form((uh2 - u_ex2)**2 * ufl.dx)), op=MPI.SUM))
error_max = domain.comm.allreduce(np.max(np.abs(uh2.x.array - u_ex2.x.array)), op=MPI.MAX)
if domain.comm.rank == 0:
    print(f"L2-error: {error_L2:.2e}, Error_max: {error_max:.2e}")

In [ ]:
udiff = fem.Function(V1)
udiff.x.petsc_vec.setArray(np.abs(uh1.x.petsc_vec.getArray() - u_ex1.x.petsc_vec.getArray()))

In [ ]:
triang = Triangulation(points[:, 0], points[:, 1], cells)

plt.figure(figsize=(8, 6))
tripcolor_plot = plt.tripcolor(triang, uh1.x.petsc_vec.getArray(), shading='gouraud')
plt.triplot(triang, color='black', linewidth=0.5)
plt.gca().set_aspect('equal')
plt.title("uh")
plt.xlabel("$x_1$")
plt.ylabel("$x_2$")
plt.colorbar(tripcolor_plot)
plt.show()

plt.figure(figsize=(8, 6))
tripcolor_plot = plt.tripcolor(triang, u_ex1.x.petsc_vec.getArray(), shading='gouraud')
plt.triplot(triang, color='black', linewidth=0.5)
plt.gca().set_aspect('equal')
plt.title("u_exact")
plt.xlabel("$x_1$")
plt.ylabel("$x_2$")
plt.colorbar(tripcolor_plot)
plt.show()

plt.figure(figsize=(8, 6))
tripcolor_plot = plt.tripcolor(triang, udiff.x.petsc_vec.getArray(), shading='gouraud')
plt.triplot(triang, color='black', linewidth=0.5)
plt.gca().set_aspect('equal')
plt.title("udiff")
plt.xlabel("$x_1$")
plt.ylabel("$x_2$")
plt.colorbar(tripcolor_plot)
plt.show()

In [ ]:
drchlt_bool = np.zeros(points.shape[0])
neumnn_bool = np.zeros(points.shape[0])

dofs_bottom_b = fem.locate_dofs_topological(V1, fdim, facet_tags.find(bottom_tag_num))
dofs_left_b = fem.locate_dofs_topological(V1, fdim, facet_tags.find(left_tag_num))
dofs_top_b = fem.locate_dofs_topological(V1, fdim, facet_tags.find(top_tag_num))
dofs_right_b = fem.locate_dofs_topological(V1, fdim, facet_tags.find(right_tag_num))

dofs_drchlt_BC = np.concatenate([dofs_left_b, dofs_right_b])
dofs_neumnn_BC = np.concatenate([dofs_bottom_b, dofs_top_b])
drchlt_bool[dofs_drchlt_BC] = 1.
neumnn_bool[dofs_neumnn_BC] = 1.
data = np.concatenate(
    [points[:, 0].reshape((-1, 1)),
     points[:, 1].reshape((-1, 1)),
     drchlt_bool.reshape((-1, 1)),
     neumnn_bool.reshape((-1, 1)),
     uh1.x.petsc_vec.getArray().reshape((-1, 1))], axis=1)

data_folder = Path("./data")
data_folder.mkdir(exist_ok=True, parents=True)
filename = data_folder / f"case_ID_{case_ID}"

np.save(filename, data)

In [ ]:
from dolfinx import io
results_folder = Path("./paraview") / f"case_ID_{case_ID}"
results_folder.mkdir(exist_ok=True, parents=True)
filename = results_folder / f"case_ID_{case_ID}"

with io.XDMFFile(domain.comm, filename.with_suffix(".xdmf"), "w") as xdmf:
    xdmf.write_mesh(domain)
    xdmf.write_function(udiff)